In [2]:
import csv

with open('data.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
print(data[0])
data.pop(0)
print(data[:10])

['id', 'video_id', 'frame_id', 'sensor_z_score', 'sensor_min_max', 'predicted', 'ground_truth']
[['0_650', '0', '650', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_675', '0', '675', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_700', '0', '700', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_725', '0', '725', '-0.9937692595422224', '0.026666666666666672', 'False', 'False'], ['0_1600', '0', '1600', '-1.0813170675566173', '0.013333333333333336', 'False', 'True'], ['0_1625', '0', '1625', '-0.9937692595422224', '0.026666666666666672', 'False', 'False'], ['0_1650', '0', '1650', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_1675', '0', '1675', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_1700', '0', '1700', '-1.1688648755710123', '0.0', 'False', 'False'], ['0_1725', '0', '1725', '-1.1688648755710123', '0.0', 'False', 'False']]


In [14]:
tp = tn = fp = fn = 0

for x in data:
    video_id = int(x[0].split('_')[0])
    if video_id < 2:
        continue
    predicted = True if x[5] == 'True' else False
    ground_truth = True if x[6] == 'True' else False
    if predicted == ground_truth:
        if ground_truth:
            tp += 1
        else:
            tn += 1
    else:
        if predicted:
            fp += 1
        else:
            fn += 1

print(tp, tn, fp, fn)

239 225 24 936


In [15]:
precision = tp / (tp + fp) # pozytywnie wykryte wśród wszystkich pozytywnych
recall = tp / (tp + fn) #daje dużo false negative - co jest złe dla wykrywania pojazdów
print(precision, recall)

0.908745247148289 0.20340425531914894


In [16]:
from keras import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [17]:
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(640, 640, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
model.load_weights('models/clasiffication_custom_augmentation_640_val.h5')

In [ ]:
datagen = ImageDataGenerator(rescale=1.0 / 255.0, validation_split=0.2)
valid_it = datagen.flow_from_directory('ground_truth_small/test', class_mode='binary', batch_size=16, target_size=(640, 640), subset='validation')